# BGC-Argo Quality Control

This is a jupyter notebook to demonstrate the fucntionality of the `bgcArgo` python module, which is based on the SOCCOM set of matlab code for performing the same task. In order to perform these tasks, WOA, NCEP, and Argo data must be available. These must be downloaded to your local machine (which can be done using `bgcArgo`, shown below). Plan to add functionality to get the necessary data remotely. 

In [ ]:
%matplotlib inline
from ipywidgets import interactive, fixed

from pathlib import Path
from bgcArgo import sprof

Set up appropriate local directories:

In [ ]:
datadir   = Path('/Users/gordonc/Documents/data')
woa_path  = datadir / 'WOA18'
ncep_path = datadir / 'NCEP'
argo_path = datadir / 'Argo'

You can tell `bgcArgo` where your local directories are set up, or specify the path to each function:

In [ ]:
sprof.set_dirs(woa_path=woa_path, ncep_path=ncep_path, argo_path=argo_path)

If the data is not already saved to your local machine, you can use the module to download the reference data as well as Argo data:
```python
import bgcArgo

# get the reference data
bgcArgo.io.get_woa18('O2sat', local_path=woa_path)
bgcArgo.io.get_ncep('pres', local_path=ncep_path)

# get argo data for float 4902481, could provide more float numbers
dacdir = '/ifremer/argo/dac/meds'
wmo_numbers = [4902481]
bgcArgo.io.get_argo(dac_path, wmo_numbers, local_path=argo_path)
```
With reference data downloaded, float data is easily loaded into an object that will have fields for each float variable:

In [ ]:
flt = sprof(4902481)

This creates a `class argo` which has various methods attached to it for visualization and QC. Example of plotting profiles:

In [ ]:
w = interactive(flt.plot, kind=fixed('profiles'), 
                varlist=[('T and O2',['TEMP', 'DOXY']),
                         ('Physics',['TEMP', 'PSAL', 'PDEN'])],
                Ncycle=(1, flt.N_CYCLES), Nprof=(1, flt.N_CYCLES), zvar=['PRES', 'PDEN'])
display(w)

time section: 

In [ ]:
w = interactive(flt.plot, kind=fixed('cscatter'), varname=['TEMP','PSAL','DOXY'], ylim=fixed((0,500)))
display(w)

Can calculate oxygen gain using WOA or NCEP data as reference values:

In [ ]:
woa_gains  = flt.calc_gains(ref='WOA')
ncep_gains = flt.calc_gains(ref='NCEP')

print('WOA Gains:', woa_gains, '\n')
print('NCEP Gains: ', ncep_gains)

Results can be plotted:

In [ ]:
w = interactive(flt.plot, kind=fixed('gain'), ref=['WOA', 'NCEP'])
display(w)

The `argo` object allows for easy plotting and data access, but if you just want to load the data from the float for your own use, that can easily be done as well:

In [ ]:
# get a python dictionary
float_dict = flt.to_dict()
# create a pandas dataframe
float_df = flt.to_dataframe()
print(float_df.head())

Next steps:
* More validation against SOCCOM matlab code
* Validation against already DMQC'ed Argo floats
* Right now works with Argo synthetic profiles (and BRtraj for in-air data) - will probably want to build individual profile functionality as well
* Optode response time correction
* Access data remotely - both reference and Argo data sets
* Have it work well with BODC version of OWC code?